<a href="https://colab.research.google.com/github/yashmathuria0001/Stock-Market-Agent/blob/main/stockagent_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install langchain langchain-community groq yfinance langchain_groq tavily-python --quiet

In [9]:
!pip install -qU duckduckgo-search

In [10]:
from google.colab import userdata
from langchain_groq import ChatGroq


llm = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get('new_api'))

In [11]:
import os
import yfinance as yf
from langchain.tools import Tool, DuckDuckGoSearchRun
from langchain.agents import initialize_agent, AgentType
import ast
import operator as op

def get_stock_price(ticker: str) -> str:
    """
    Fetches the latest stock price using Yahoo Finance.
    """
    try:
        stock = yf.Ticker(ticker)
        price = stock.history(period="1d")["Close"].iloc[-1]
        return f"The latest stock price of {ticker} is ${price:.2f}"
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_finance_report(ticker: str) -> str:
    """
    Fetches key financial data for a given ticker symbol using yfinance.
    Returns basic company info and a snapshot of recent financials.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        financials = stock.financials
        report = f"Company: {info.get('longName', ticker)}\n"
        report += f"Sector: {info.get('sector', 'N/A')}\n"
        report += "Recent Financials:\n"
        report += str(financials.head())
        return report
    except Exception as e:
        return f"Error fetching finance report for {ticker}: {str(e)}"

finance_report_tool = Tool(
    name="Yahoo Finance Report",
    func=get_finance_report,
    description="Fetches key financial data for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)


web_search_tool = DuckDuckGoSearchRun()

operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.USub: op.neg,
}

def safe_eval(expr: str) -> float:
    """
    Safely evaluate an arithmetic expression using Python's AST.
    """
    def eval_node(node):
        if isinstance(node, ast.Num):
            return node.n
        elif isinstance(node, ast.BinOp):
            return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
        elif isinstance(node, ast.UnaryOp):
            return operators[type(node.op)](eval_node(node.operand))
        else:
            raise TypeError(f"Unsupported type: {node}")

    node = ast.parse(expr, mode='eval').body
    return eval_node(node)

def simple_calculator(expression: str) -> str:
    """
    Evaluates a simple arithmetic expression safely.
    Example: "45 * 67"
    """
    try:
        result = safe_eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

calculator_tool = Tool(
    name="Custom Calculator",
    func=simple_calculator,
    description="Evaluates basic arithmetic expressions safely. Example: '45 * 67'."
)

stock_price_tool = Tool(
    name="Stock Price Fetcher",
    func=get_stock_price,
    description="Fetches the latest stock price for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)
agent = initialize_agent(
    tools=[stock_price_tool, calculator_tool, finance_report_tool, web_search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


<ipython-input-11-64d9d1011677>:93: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
var="What is the range of buy and sell for Nvida stock?"
print(agent.run(var))

<ipython-input-12-c7b38b29decc>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run(var))




> Entering new AgentExecutor chain...
Thought: I need to get the current stock price of NVIDIA to determine the range of buy and sell prices.

Action: Stock Price Fetcher
Action Input: NVDA
Observation: The latest stock price of NVDA is $121.83
Thought:Thought: I also need to get the key financial data of NVIDIA to determine the range of buy and sell prices.

Action: Yahoo Finance Report
Action Input: NVDA
Observation: Company: NVIDIA Corporation
Sector: Technology
Recent Financials:
                                           2025-01-31     2024-01-31  \
Tax Effect Of Unusual Items                       0.0            0.0   
Tax Rate For Calcs                              0.133           0.12   
Normalized EBITDA                       86137000000.0  35583000000.0   
Total Unusual Items                               0.0            0.0   
Total Unusual Items Excluding Goodwill            0.0            0.0   

                                          2023-01-31     2022-01-31  
Tax Ef